In [1]:
import os.path

import numpy as np
import librosa
from preprocess.process_utils import get_file_dict, wav_split_overlap
import torch
import torchaudio
from tqdm import tqdm


def MODMA_P(path: str, duration=10, resample_rate=16000, threshold=1, overlap=2.5):
    """
    使用torch读取音频数据，加速读取 推荐使用，（将小于10s的短音频补零至10s，将大于10s的音频分成若干段(包含重叠)）
    Args:
        threshold: 对于多出来的部分，大于threshold补零，小于threshold截断
        order: [1,2,3] 1:13阶MFCC 2:13阶MFCC+一阶差分 3:13阶MFCC+一阶差分+二阶差分
        extra: 是否保存分段信息(MODMA)
        path: 音频文件存放路径
        code: one-hot编码
        frame_length: 帧长
        duration: 划分时长
        resample_rate: 重采样
        overlap: 重叠时长

    Returns:

    """
    dataset = path.split('/')[-1]
    print(f"fastV1: {dataset}")
    wav_dict = get_file_dict(path=path)
    extra_info = {}
    sample_wav = wav_dict[list(wav_dict.keys())[0]][0]
    _, sr = librosa.load(sample_wav, sr=None)
    expect_length = sr * duration
    for key in (wav_dict.keys()):
        print(f"process: {key}")
        slice_num = 0
        stack_data = None
        for wav in tqdm(wav_dict[key]):
            data, sr = torchaudio.load(wav)
            actual_length = data.shape[1]
            if actual_length < expect_length:
                data = torch.cat([data, torch.zeros([1, expect_length - actual_length])], dim=1)
                slice_num += 1
            elif actual_length > expect_length:
                data, split_num = wav_split_overlap(data, sr, threshold, duration, overlap)
                slice_num += split_num
            else:
                slice_num += 1
            if stack_data is None:
                stack_data = data
            else:
                stack_data = torch.cat([stack_data, data], dim=0)
        extra_info[key] = slice_num
        stack_data = stack_data.numpy().astype(np.float32)
        if sr != resample_rate:
            stack_data = librosa.resample(stack_data, orig_sr=sr, target_sr=resample_rate)
        stack_data = torch.Tensor(stack_data)
        for i in range(stack_data.shape[0]):
            if not os.path.exists(f"x/MODMA/{key}/"):
                os.mkdir(f"x/MODMA/{key}/")
            index = "{:0>3d}".format(i)
            torchaudio.save(f"x/MODMA/{key}/{index}.wav",stack_data[i].unsqueeze(0), resample_rate)

MODMA_P(path="../preprocess/MODMA/")


fastV1: 
process: 02010001


100%|██████████| 29/29 [00:00<00:00, 279.78it/s]


process: 02010002


100%|██████████| 29/29 [00:00<00:00, 148.79it/s]


process: 02010003


...

100%|██████████| 29/29 [00:00<00:00, 92.67it/s] 


process: 02030016


100%|██████████| 29/29 [00:00<00:00, 105.81it/s]


process: 02030017


100%|██████████| 29/29 [00:00<00:00, 82.89it/s]


In [2]:
def CASIA_P(path: str, duration=6, resample_rate=16000):
    """
    使用torch读取音频数据，加速读取 推荐使用，（将小于10s的短音频补零至10s，将大于10s的音频分成若干段(包含重叠)）
    Args:
        threshold: 对于多出来的部分，大于threshold补零，小于threshold截断
        order: [1,2,3] 1:13阶MFCC 2:13阶MFCC+一阶差分 3:13阶MFCC+一阶差分+二阶差分
        extra: 是否保存分段信息(MODMA)
        path: 音频文件存放路径
        code: one-hot编码
        frame_length: 帧长
        duration: 划分时长
        resample_rate: 重采样
        overlap: 重叠时长

    Returns:

    """
    dataset = path.split('/')[-1]
    print(f"fastV1: {dataset}")
    wav_dict = get_file_dict(path=path)
    sample_wav = wav_dict[list(wav_dict.keys())[0]][0]
    _, sr = librosa.load(sample_wav, sr=None)
    expect_length = sr * duration
    for key in (wav_dict.keys()):
        print(f"process: {key}")
        stack_data = None
        for wav in tqdm(wav_dict[key]):
            data, sr = torchaudio.load(wav)
            actual_length = data.shape[1]
            if actual_length < expect_length:
                data = torch.cat([data, torch.zeros([1, expect_length - actual_length])], dim=1)
            elif actual_length > expect_length:
                data = data[:, :expect_length]
            if stack_data is None:
                stack_data = data
            else:
                stack_data = torch.cat([stack_data, data], dim=0)
        stack_data = stack_data.numpy().astype(np.float32)
        if sr != resample_rate:
            stack_data = librosa.resample(stack_data, orig_sr=sr, target_sr=resample_rate)
        stack_data = torch.Tensor(stack_data)
        for i in range(stack_data.shape[0]):
            if not os.path.exists(f"x/CASIA/{key}/"):
                os.mkdir(f"x/CASIA/{key}/")
            index = "{:0>4d}".format(i)
            torchaudio.save(f"x/CASIA/{key}/{index}.wav",stack_data[i].unsqueeze(0), resample_rate)
CASIA_P(path="../preprocess/datasets/CASIA")

fastV1: CASIA
process: anger


100%|██████████| 1200/1200 [00:50<00:00, 23.84it/s]


process: fear


100%|██████████| 1200/1200 [00:51<00:00, 23.43it/s]


process: happy


...

100%|██████████| 1200/1200 [00:56<00:00, 21.37it/s]


process: sad


100%|██████████| 1200/1200 [00:59<00:00, 20.18it/s]


process: surprise


100%|██████████| 1200/1200 [00:56<00:00, 21.30it/s]
